In [1]:
# import dependencies
import pandas as pd
from bs4 import BeautifulSoup as bs
import requests
from splinter import Browser

In [2]:
# setup Splinter
executable_path = {'executable_path': 'C:/Users/ryban/Anaconda3/chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)

### NASA Mars News

In [3]:
# get the url, retrieve the page with the requests module, and create BeautifulSoup object
url = 'https://mars.nasa.gov/news/'
response = requests.get(url)
soup = bs(response.text, 'lxml')

In [4]:
# find the news article title and save to a variable: news_title
news_title = soup.find('div', class_='content_title').a.text.strip()

# display the news article title
print(news_title)

NASA's Curiosity Mars Rover Finds a Clay Cache


In [5]:
# find the news article teaser text and save to a variable: news_p
news_p = soup.find('div', class_='image_and_description_container').div.text.strip()

# display the news article teaser text
print(news_p)

The rover recently drilled two samples, and both showed the highest levels of clay ever found during the mission.


### JPL Mars Space Images - Featured Image

In [6]:
# open website and navigate to the page containing the featured image
url = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'
browser.visit(url)
browser.click_link_by_partial_text('FULL IMAGE')
browser.is_element_not_present_by_text('more info', wait_time=1)
browser.click_link_by_partial_text('more info')

In [7]:
# scrape the html for the url for the large-size image and save to a variable: featured_image_url
html = browser.html
soup = bs(html, 'html.parser')
img_link = soup.find('figure', class_='lede').a['href']
featured_image_url = 'https://www.jpl.nasa.gov'+img_link
print(featured_image_url)

https://www.jpl.nasa.gov/spaceimages/images/largesize/PIA14579_hires.jpg


### Mars Weather

In [8]:
# get the url, retrieve the page with the requests module, and create BeautifulSoup object
url = 'https://twitter.com/MarsWxReport?lang=en'
response = requests.get(url)
soup = bs(response.text, 'lxml')

In [9]:
# find the text for the latest tweet and save to a variable: mars_weather
pic_a_tag = soup.find('p', class_='TweetTextSize').a.decompose()
mars_weather = soup.find('p', class_='TweetTextSize').text

# display the text of the tweet
print(mars_weather)

InSight sol 265 (2019-08-25) low -99.4ºC (-146.9ºF) high -26.3ºC (-15.3ºF)
winds from the SSE at 5.3 m/s (12.0 mph) gusting to 16.1 m/s (35.9 mph)
pressure at 7.50 hPa


### Mars Facts

In [10]:
# url for Mars Facts webpage
url = 'https://space-facts.com/mars/'

# read html table and store to a variable
facts = pd.read_html(url)

# save the desired table to a dataframe and name the columns
facts_df = facts[1]
facts_df.columns=['description','value']
facts_df = facts_df.set_index('description')

# write the dataframe to an html table string
facts_table = facts_df.to_html()

# display the html table string
facts_table

'<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>value</th>\n    </tr>\n    <tr>\n      <th>description</th>\n      <th></th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>Equatorial Diameter:</th>\n      <td>6,792 km</td>\n    </tr>\n    <tr>\n      <th>Polar Diameter:</th>\n      <td>6,752 km</td>\n    </tr>\n    <tr>\n      <th>Mass:</th>\n      <td>6.39 × 10^23 kg (0.11 Earths)</td>\n    </tr>\n    <tr>\n      <th>Moons:</th>\n      <td>2 (Phobos &amp; Deimos)</td>\n    </tr>\n    <tr>\n      <th>Orbit Distance:</th>\n      <td>227,943,824 km (1.38 AU)</td>\n    </tr>\n    <tr>\n      <th>Orbit Period:</th>\n      <td>687 days (1.9 years)</td>\n    </tr>\n    <tr>\n      <th>Surface Temperature:</th>\n      <td>-87 to -5 °C</td>\n    </tr>\n    <tr>\n      <th>First Record:</th>\n      <td>2nd millennium BC</td>\n    </tr>\n    <tr>\n      <th>Recorded By:</th>\n      <td>Egyptian astronomers</td>\n    </tr>\

### Mars Hemispheres

In [11]:
# get the url, retrieve the page with the requests module, and create BeautifulSoup object
url = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'
response = requests.get(url)
soup = bs(response.text, 'lxml')

In [12]:
# create list of sections containing images and store to a variable: image_items
image_items = soup.find_all('div',class_='item')

# initialze list to store image title and url dictionaries
hemisphere_image_urls = []

# loop through the items and append the image title and url to the list as a dictionary
for item in image_items:
    img_title = item.h3.text    
    img_link = 'https://astrogeology.usgs.gov'+item.a['href']
    browser.visit(img_link)
    img_url = browser.find_link_by_text('Sample')['href']
    hemisphere_image_urls.append({'title':img_title,'img_url':img_url})
browser.quit()

In [13]:
# display the list of dictionaries
hemisphere_image_urls

[{'title': 'Cerberus Hemisphere Enhanced',
  'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg'},
 {'title': 'Schiaparelli Hemisphere Enhanced',
  'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg'},
 {'title': 'Syrtis Major Hemisphere Enhanced',
  'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg'},
 {'title': 'Valles Marineris Hemisphere Enhanced',
  'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg'}]

In [14]:
# store all data to a dictionary
scraped_data = {'news_title':news_title,'news_p':news_p,'featured_image_url':featured_image_url,
                'mars_weather':mars_weather,'facts_table':facts_table,
                'hemisphere_image_urls':hemisphere_image_urls}
# display the dictionary
scraped_data

{'news_title': "NASA's Curiosity Mars Rover Finds a Clay Cache",
 'news_p': 'The rover recently drilled two samples, and both showed the highest levels of clay ever found during the mission.',
 'featured_image_url': 'https://www.jpl.nasa.gov/spaceimages/images/largesize/PIA14579_hires.jpg',
 'mars_weather': 'InSight sol 265 (2019-08-25) low -99.4ºC (-146.9ºF) high -26.3ºC (-15.3ºF)\nwinds from the SSE at 5.3 m/s (12.0 mph) gusting to 16.1 m/s (35.9 mph)\npressure at 7.50 hPa',
 'facts_table': '<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>value</th>\n    </tr>\n    <tr>\n      <th>description</th>\n      <th></th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>Equatorial Diameter:</th>\n      <td>6,792 km</td>\n    </tr>\n    <tr>\n      <th>Polar Diameter:</th>\n      <td>6,752 km</td>\n    </tr>\n    <tr>\n      <th>Mass:</th>\n      <td>6.39 × 10^23 kg (0.11 Earths)</td>\n    </tr>\n    <tr>\n      <th>Moons: